In [1]:
import os
import argparse
import yaml
import aniposelib
import cv2
import numpy as np
import pandas as pd

import plotly.express as px

In [2]:
%load_ext autoreload
%autoreload 2

In [2]:
board = aniposelib.boards.CharucoBoard(squaresX=6,
                                            squaresY=6,
                                            square_length=24.33,
                                            marker_length=17,
                                            marker_bits=5,
                                            dict_size=50)

In [2]:
base_path = "/Users/etytel01/Documents/2025/triangulate"

calibration_videos = ["raw data/CharucoPostCalib_LateralOrtho.mp4", 
                      "raw data/CharucoPostCalib_LateralOffAxis.mp4"]
camera_names = ["ortho", "offaxis"]

In [4]:
camgroup = aniposelib.cameras.CameraGroup.from_names(camera_names)

In [5]:
vidnames = [[os.path.join(base_path, vid)] for vid in calibration_videos]

In [6]:
err, rows = camgroup.calibrate_videos(vidnames, board, 
                                init_intrinsics=True, init_extrinsics=True, 
                                verbose=True)


/Users/etytel01/Documents/2025/triangulate/raw data/CharucoPostCalib_LateralOrtho.mp4


100%|█████████████████████████████████| 50/50 [00:25<00:00,  1.99it/s]


50 boards detected
/Users/etytel01/Documents/2025/triangulate/raw data/CharucoPostCalib_LateralOffAxis.mp4


100%|█████████████████████████████████| 50/50 [00:11<00:00,  4.17it/s]


50 boards detected
[{'name': 'ortho', 'size': [2560, 904], 'matrix': [[4248.632751205015, 0.0, 1279.5], [0.0, 4248.632751205015, 451.5], [0.0, 0.0, 1.0]], 'distortions': [0.0, 0.0, 0.0, 0.0, 0.0], 'rotation': [0.0, 0.0, 0.0], 'translation': [0.0, 0.0, 0.0]}, {'name': 'offaxis', 'size': [2560, 904], 'matrix': [[23263.575037121605, 0.0, 1279.5], [0.0, 23263.575037121605, 451.5], [0.0, 0.0, 1.0]], 'distortions': [0.0, 0.0, 0.0, 0.0, 0.0], 'rotation': [0.0, 0.0, 0.0], 'translation': [0.0, 0.0, 0.0]}]
defaultdict(<class 'int'>, {('ortho', 'offaxis'): 50, ('offaxis', 'ortho'): 50})


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


error:  3.5981582779147416
n_samples: 200
{(0, 1): (1000, array([0.77148755, 8.47288616]))}
error: 3.67, mu: 8.5, ratio: 0.750
   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         1.8352e+04                                    7.03e+05    
       1              3         4.7878e+03      1.36e+04       1.25e+03       8.47e+05    
       2              5         1.5518e+03      3.24e+03       3.57e+02       3.06e+05    
       3              7         1.0204e+03      5.31e+02       1.72e+02       2.35e+05    
       4              8         8.4647e+02      1.74e+02       1.35e+02       3.35e+05    
       5              9         6.8705e+02      1.59e+02       1.51e+02       3.15e+05    
       6             10         5.4037e+02      1.47e+02       1.42e+02       3.25e+05    
       7             11         3.7064e+02      1.70e+02       1.14e+02       4.85e+04    
       8             12         3.3638e+02      3.43e+

In [7]:
import pandas as pd

In [8]:
df_all = []
for cam1, rows_cam in zip(camera_names, rows):
    df_cam = []
    col_idx = pd.MultiIndex.from_product([[cam1], ['x', 'y']],
                                         names = ['camera', 'var'])
    for pts1 in rows_cam:
        fr1 = pts1['framenum'][1]
        row_idx = pd.MultiIndex.from_product([[fr1], pts1['ids'][:,0]], 
                                             names=['frame', 'id'])
        df_cam.append(pd.DataFrame(data = np.squeeze(pts1['corners']), 
                                   index=row_idx, 
                                   columns=col_idx))
    df_all.append(pd.concat(df_cam))

df_all = pd.concat(df_all, axis=1)

In [9]:
df_all

camera          ortho                  offaxis            
var                 x           y            x           y
frame id                                                  
0     0   2400.616455  297.263306  1884.004272  373.914978
      1   2404.097412  390.699310  1883.046143  477.999939
      2   2407.309082  483.782867  1882.135010  581.918701
      3   2410.435303  576.601501  1881.114624  685.594116
      4   2413.236328  669.627136          NaN         NaN
...               ...         ...          ...         ...
15    6           NaN         NaN  1554.092896  147.297440
      11          NaN         NaN  1462.564209  142.629135
25    6           NaN         NaN  1257.937866  144.787567
      11          NaN         NaN  1172.665649  139.786285
31    20          NaN         NaN  1173.201538  117.983902

[1116 rows x 4 columns]

In [10]:
df_all.loc[:,('ortho',slice(None))].to_numpy().shape

(1116, 2)

In [11]:
ptsmatrix = []
for cam1 in camera_names:
    pts1 = df_all.loc[:, (cam1, slice(None))].to_numpy()
    ptsmatrix.append(pts1[np.newaxis, :,:])

In [12]:
ptsmatrix = np.concatenate(ptsmatrix, axis=0)

In [13]:
pts3d = camgroup.triangulate(ptsmatrix, progress=True, undistort=True)

100%|███████████████████████████| 1116/1116 [00:00<00:00, 3763.71it/s]


In [14]:
col_idx = pd.MultiIndex.from_product([['3D'], ['x', 'y', 'z']],
                                        names=['camera','var'])
dfpts3d = pd.DataFrame(pts3d, index=df_all.index, 
                            columns=col_idx)
pts = pd.concat((df_all, dfpts3d), axis=1)


In [15]:
pts

camera          ortho                  offaxis                      3D  \
var                 x           y            x           y           x   
frame id                                                                 
0     0   2400.616455  297.263306  1884.004272  373.914978  308.228909   
      1   2404.097412  390.699310  1883.046143  477.999939  309.388765   
      2   2407.309082  483.782867  1882.135010  581.918701  310.650615   
      3   2410.435303  576.601501  1881.114624  685.594116  312.004177   
      4   2413.236328  669.627136          NaN         NaN         NaN   
...               ...         ...          ...         ...         ...   
15    6           NaN         NaN  1554.092896  147.297440         NaN   
      11          NaN         NaN  1462.564209  142.629135         NaN   
25    6           NaN         NaN  1257.937866  144.787567         NaN   
      11          NaN         NaN  1172.665649  139.786285         NaN   
31    20          NaN         NaN  1173.201538  117.983902         NaN   

camera                           
var               y           z  
frame id                         
0     0  -43.993394  886.597423  
      1  -18.420885  886.530870  
      2    7.074293  886.488919  
      3   32.535340  886.348333  
      4         NaN         NaN  
...             ...         ...  
15    6         NaN         NaN  
      11        NaN         NaN  
25    6         NaN         NaN  
      11        NaN         NaN  
31    20        NaN         NaN  

[1116 rows x 7 columns]

In [16]:
reproj = camgroup.project(pts3d)
reproj = reproj.transpose((1,0,2))
reproj = reproj.reshape((-1, 2*len(camera_names)))

In [17]:
col_idx = pd.MultiIndex.from_product([camgroup.get_names(), ['xr', 'yr']],
                                        names=['camera','var'])

reproj = pd.DataFrame(reproj, 
                        index=pts.index, 
                        columns=col_idx)

pts = pd.concat((pts, reproj), axis=1)


In [18]:
pts

camera          ortho                  offaxis                      3D  \
var                 x           y            x           y           x   
frame id                                                                 
0     0   2400.616455  297.263306  1884.004272  373.914978  308.228909   
      1   2404.097412  390.699310  1883.046143  477.999939  309.388765   
      2   2407.309082  483.782867  1882.135010  581.918701  310.650615   
      3   2410.435303  576.601501  1881.114624  685.594116  312.004177   
      4   2413.236328  669.627136          NaN         NaN         NaN   
...               ...         ...          ...         ...         ...   
15    6           NaN         NaN  1554.092896  147.297440         NaN   
      11          NaN         NaN  1462.564209  142.629135         NaN   
25    6           NaN         NaN  1257.937866  144.787567         NaN   
      11          NaN         NaN  1172.665649  139.786285         NaN   
31    20          NaN         NaN  1173.201538  117.983902         NaN   

camera                                 ortho                  offaxis  \
var               y           z           xr          yr           xr   
frame id                                                                
0     0  -43.993394  886.597423  2400.632604  297.499522  1883.994871   
      1  -18.420885  886.530870  2404.099812  390.743723  1883.045601   
      2    7.074293  886.488919  2407.308978  483.781534  1882.134984   
      3   32.535340  886.348333  2410.436233  576.633843  1881.115989   
      4         NaN         NaN          NaN         NaN          NaN   
...             ...         ...          ...         ...          ...   
15    6         NaN         NaN          NaN         NaN          NaN   
      11        NaN         NaN          NaN         NaN          NaN   
25    6         NaN         NaN          NaN         NaN          NaN   
      11        NaN         NaN          NaN         NaN          NaN   
31    20        NaN         NaN          NaN         NaN          NaN   

camera                
var               yr  
frame id              
0     0   373.650383  
      1   477.950249  
      2   581.920181  
      3   685.558039  
      4          NaN  
...              ...  
15    6          NaN  
      11         NaN  
25    6          NaN  
      11         NaN  
31    20         NaN  

[1116 rows x 11 columns]

In [19]:
for cam1 in camera_names:
    detect1 = pts.loc[:, (cam1, ('x', 'y'))].to_numpy()
    reproj1 = pts.loc[:, (cam1, ('xr', 'yr'))].to_numpy()

    err = np.linalg.norm(reproj1 - detect1, axis=1)

    pts[(cam1, "err")] = pd.Series(err, index=pts.index)



In [20]:
pts

camera          ortho                  offaxis                      3D  \
var                 x           y            x           y           x   
frame id                                                                 
0     0   2400.616455  297.263306  1884.004272  373.914978  308.228909   
      1   2404.097412  390.699310  1883.046143  477.999939  309.388765   
      2   2407.309082  483.782867  1882.135010  581.918701  310.650615   
      3   2410.435303  576.601501  1881.114624  685.594116  312.004177   
      4   2413.236328  669.627136          NaN         NaN         NaN   
...               ...         ...          ...         ...         ...   
15    6           NaN         NaN  1554.092896  147.297440         NaN   
      11          NaN         NaN  1462.564209  142.629135         NaN   
25    6           NaN         NaN  1257.937866  144.787567         NaN   
      11          NaN         NaN  1172.665649  139.786285         NaN   
31    20          NaN         NaN  1173.201538  117.983902         NaN   

camera                                 ortho                  offaxis  \
var               y           z           xr          yr           xr   
frame id                                                                
0     0  -43.993394  886.597423  2400.632604  297.499522  1883.994871   
      1  -18.420885  886.530870  2404.099812  390.743723  1883.045601   
      2    7.074293  886.488919  2407.308978  483.781534  1882.134984   
      3   32.535340  886.348333  2410.436233  576.633843  1881.115989   
      4         NaN         NaN          NaN         NaN          NaN   
...             ...         ...          ...         ...          ...   
15    6         NaN         NaN          NaN         NaN          NaN   
      11        NaN         NaN          NaN         NaN          NaN   
25    6         NaN         NaN          NaN         NaN          NaN   
      11        NaN         NaN          NaN         NaN          NaN   
31    20        NaN         NaN          NaN         NaN          NaN   

camera                   ortho   offaxis  
var               yr       err       err  
frame id                                  
0     0   373.650383  0.236768  0.264762  
      1   477.950249  0.044477  0.049693  
      2   581.920181  0.001337  0.001481  
      3   685.558039  0.032355  0.036103  
      4          NaN       NaN       NaN  
...              ...       ...       ...  
15    6          NaN       NaN       NaN  
      11         NaN       NaN       NaN  
25    6          NaN       NaN       NaN  
      11         NaN       NaN       NaN  
31    20         NaN       NaN       NaN  

[1116 rows x 13 columns]

In [21]:
from copy import copy

ptsf = copy(pts)
ptsf.columns = ['_'.join(a) for a in ptsf.columns.to_flat_index()]
ptsf = ptsf.reset_index()


In [22]:
px.histogram(ptsf, x="ortho_err")

In [29]:
ptsf = pts.loc[:, (camera_names, slice(None))].stack(level=0).reset_index()

/var/folders/rs/r2xck00j2td2g2w0g6s_t2lx370bnk/T/ipykernel_17019/1366755653.py:1: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



In [30]:
px.histogram(ptsf, x="err", facet_col="camera")

In [7]:
f1 = "raw data/PreliminaryAccelerations.000_PS21_Trial1_LateralOffAxis.analysis.csv"
f2 = "raw data/PreliminaryAccelerations.001_PS21_Trial1_LateralOrtho.analysis.csv"

In [8]:
data1 = pd.read_csv(f1)
data2 = pd.read_csv(f2)

In [12]:
data1.head()

,track,frame_idx,instance.score,Snout.x,Snout.y,Snout.score,LeftEye.x,LeftEye.y,LeftEye.score,RightEye.x,...,SoftDorsalBase.score,SoftDorsalFrontTip.x,SoftDorsalFrontTip.y,SoftDorsalFrontTip.score,SoftDorsalBackTip.x,SoftDorsalBackTip.y,SoftDorsalBackTip.score,AnalTip.x,AnalTip.y,AnalTip.score
0,NaN,0,11.398748,1664.259888,816.474365,0.897653,1704.499634,799.299805,0.786543,NaN,...,0.819831,1983.611694,720.251892,0.589965,2040.490479,792.533508,0.558482,2047.878906,937.515625,0.486594
1,NaN,1,11.337370,1664.183350,816.326355,0.916883,1704.616333,793.324280,0.802946,NaN,...,0.808387,1983.721924,720.231445,0.570687,2040.408447,792.575806,0.551887,2047.840088,937.526245,0.477825
2,NaN,2,11.363713,1664.050293,816.305603,0.917555,1704.527954,793.296570,0.810382,NaN,...,0.809449,1983.764893,720.192200,0.575877,2040.401001,792.486877,0.547943,2047.875610,937.535034,0.471660
3,NaN,3,11.169652,1663.940308,816.366394,0.895664,1704.428101,793.190796,0.789035,NaN,...,0.798446,1983.607178,720.136902,0.568006,2040.266235,792.486877,0.539308,2047.762573,937.546143,0.459907
4,NaN,4,11.270613,1663.866699,816.219788,0.933337,1704.379272,793.262756,0.826786,NaN,...,0.794351,1983.968750,720.087402,0.558724,2040.300781,792.529541,0.534846,2047.848511,937.577209,0.454737


In [9]:
from label3d.triangulate import reorganize_sleap_csv
data1 = reorganize_sleap_csv(data1)
data2 = reorganize_sleap_csv(data2)

In [21]:
camdata = pd.concat((data1, data2), axis = 1, keys = ['d1', 'd2'], names = ['camera', 'var'])
camdata = pd.concat((camdata,), axis=0, keys=[1,], names=['trial'])

In [22]:
camdata

camera                                       d1                        \
var                                           x           y     score   
trial frame_idx point                                                   
1     0         Snout               1664.259888  816.474365  0.897653   
                LeftEye             1704.499634  799.299805  0.786543   
                RightEye                    NaN         NaN  0.000000   
                PeduncleTop         2080.956055  832.108093  0.606636   
                PeduncleMid         2096.571533  864.598816  0.612015   
...                                         ...         ...       ...   
      404       AnalBase             513.820373  777.823143       NaN   
                SoftDorsalBase       473.046752  620.665857       NaN   
                SoftDorsalFrontTip   476.897319  586.194955       NaN   
                SoftDorsalBackTip    536.848509  613.564285       NaN   
                AnalTip                     NaN         NaN       NaN   

camera                                                      d2              \
var                                instance.score            x           y   
trial frame_idx point                                                        
1     0         Snout                   11.398748  2159.782227  719.849976   
                LeftEye                 11.398748  2216.075195  688.339111   
                RightEye                11.398748  2496.093750  704.790771   
                PeduncleTop             11.398748          NaN         NaN   
                PeduncleMid             11.398748          NaN         NaN   
...                                           ...          ...         ...   
      404       AnalBase                      NaN   761.084412  784.104797   
                SoftDorsalBase                NaN   704.282654  624.381409   
                SoftDorsalFrontTip            NaN   721.069397  585.291870   
                SoftDorsalBackTip             NaN   800.908630  632.024109   
                AnalTip                       NaN          NaN         NaN   

camera                                                       
var                                    score instance.score  
trial frame_idx point                                        
1     0         Snout               0.635276       4.898228  
                LeftEye             0.324416       4.898228  
                RightEye            0.202763       4.898228  
                PeduncleTop         0.000000       4.898228  
                PeduncleMid         0.000000       4.898228  
...                                      ...            ...  
      404       AnalBase            0.585093      12.871907  
                SoftDorsalBase      1.215951      12.871907  
                SoftDorsalFrontTip  1.200003      12.871907  
                SoftDorsalBackTip   0.806408      12.871907  
                AnalTip             0.000000      12.871907  

[8100 rows x 8 columns]

In [28]:
pts = camdata.loc[:, (['d1', 'd2'], ['x','y'])]

# triangulate to 3D
ptsmatrix = pts.to_numpy().reshape((-1, 2, 2))
ptsmatrix = ptsmatrix.transpose((1,0,2)).astype(np.float64)


In [29]:
ptsmatrix.shape

(2, 8100, 2)